In [2]:
import pandas as pd

from datetime import datetime
import time

import sys
sys.path.append('../../')
sys.path.append('../../../')

In [3]:
from golf_course_project.pygolfdata.data import shotlink

pd.options.display.max_rows = 200

d = shotlink.get_combined_data_from_file('golf_course_project_data/combined_shots_and_weather_2016_2016.csv')
d.shape

(1141966, 65)

In [4]:
# Drop columns which have more than 1% of their values missing
# Then drop rows missing values

pct_threshold = 0.01
no = d.apply(lambda x: x.isna().sum() / d.shape[0])
drop_these_lame_columns = no[no >= pct_threshold].index

dnew = d.drop(columns=drop_these_lame_columns)

#dnew = dnew.dropna(axis=0, how='any')

In [5]:
dnew.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1141966 entries, 0 to 1141965
Data columns (total 52 columns):
TourCode                   1141966 non-null category
TourDescription            1141966 non-null category
Year                       1141966 non-null uint16
TournamentNum              1141966 non-null uint16
PlayerNum                  1141966 non-null uint16
CourseNum                  1141966 non-null uint16
PermanentTournamentNum     1141966 non-null uint16
PlayerFirstName            1141966 non-null category
PlayerLastName             1141966 non-null category
Round                      1141966 non-null uint8
TournamentName             1141966 non-null category
CourseName_shots           1141966 non-null object
Hole                       1141966 non-null uint8
HoleScore                  1141601 non-null float32
ParValue                   1141966 non-null uint8
Yardage                    1141966 non-null uint16
Shot                       1141966 non-null uint8
ShotType     

In [6]:
dnew = d.dropna(axis=1, how='any')
dnew.columns

Index(['TourCode', 'TourDescription', 'Year', 'TournamentNum', 'PlayerNum',
       'CourseNum', 'PermanentTournamentNum', 'PlayerFirstName',
       'PlayerLastName', 'Round', 'TournamentName', 'CourseName_shots', 'Hole',
       'ParValue', 'Yardage', 'Shot', 'ShotType', 'NumStrokes', 'Distance',
       'DistanceToPin', 'InTheHoleFlag', 'AroundTheGreenFlag',
       'DistanceToHoleAfterShot', 'Time', 'XCoordinate', 'YCoordinate',
       'ZCoordinate', 'DistanceFromCenter', 'DistanceFromEdge', 'Date_shots',
       'StrokesGainedBaseline', 'RecoveryShot', 'ShotDateAndTime',
       'PlayerName', 'Date_weather', 'Hour', 'Latitude', 'Longitude',
       'Summary', 'DegreesFahrenheit', 'Humidity', 'Visibility',
       'PrecipitationIntensity', 'CourseName_weather', 'WeatherDateAndHour',
       'TimeDifference'],
      dtype='object')

In [7]:
col_for_model = ['Summary', 'DegreesFahrenheit', 'Humidity', 'Visibility', 'PrecipitationIntensity', 'StrokesGainedBaseline']
df = dnew[col_for_model]
df = df.dropna(axis=0, how='any')
df.describe()

,DegreesFahrenheit,Humidity,Visibility,PrecipitationIntensity,StrokesGainedBaseline
count,1.141966e+06,1.141966e+06,1.141966e+06,1.141966e+06,1.141966e+06
mean,7.404843e+01,5.882784e-01,9.578472e+00,1.961086e-03,1.861721e-03
std,8.995831e+00,1.886622e-01,9.515421e-01,1.251801e-02,3.120983e-01
min,3.697000e+01,1.000000e-01,2.110000e+00,0.000000e+00,-3.206000e+00
25%,6.709000e+01,4.500000e-01,9.630000e+00,0.000000e+00,-1.100000e-01
50%,7.588000e+01,5.900000e-01,9.920000e+00,0.000000e+00,1.000000e-03
75%,8.080000e+01,7.300000e-01,1.000000e+01,0.000000e+00,1.190000e-01
max,9.275000e+01,1.000000e+00,1.000000e+01,5.134000e-01,4.752000e+00


In [11]:
from sklearn import preprocessing, linear_model
from sklearn.model_selection import train_test_split
import copy

response = 'StrokesGainedBaseline'
X = copy.deepcopy(df)
y = X[response]
X = X.drop(columns=response)
X = pd.get_dummies(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scalerX = preprocessing.StandardScaler().fit(X_train)
scalerY = preprocessing.StandardScaler().fit(y_train.reshape(-1, 1))

X_train_scaled = scalerX.transform(X_train)
X_test_scaled = scalerX.transform(X_test)
y_train_scaled = scalerY.transform(y_train.reshape(-1, 1))
y_test_scaled = scalerY.transform(y_test.reshape(-1, 1))

C:\Users\ucale\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  if sys.path[0] == '':


C:\Users\ucale\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  app.launch_new_instance()
C:\Users\ucale\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


In [13]:
from sklearn.metrics import mean_squared_error, r2_score

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train_scaled, y_train_scaled)

# Make predictions using the testing set
y_pred = regr.predict(X_test_scaled)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(y_test_scaled, y_pred))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test_scaled, y_pred))

Coefficients: 
 [[ 8.46852448e-03  1.57730065e-02  2.93760457e-03 -3.05083937e-03
  -6.47181661e+10 -5.56604798e+10 -9.01273375e+09 -1.76958803e+10
  -1.09608250e+10 -2.61683822e+09 -9.88929694e+09 -1.24589652e+10
  -2.93865696e+10 -1.20397205e+10 -1.97873793e+10 -1.91220524e+10
  -2.69689436e+09 -4.57104139e+09 -1.16113741e+10 -4.64228339e+09
  -3.43462463e+09]]
Mean squared error: 0.10
Variance score: 0.00


In [16]:
from sklearn.model_selection import KFold
from sklearn.linear_model import RidgeCV

kf = KFold(n_splits=10, shuffle=True)
alphas = (0.001, 0.003, 0.006, 0.01, 0.03, 0.06, 0.1, 0.3, 0.6, 1)
rcv = RidgeCV(alphas=alphas, cv=kf)

rcv.fit(X_train_scaled, y_train_scaled)

RidgeCV(alphas=(0.001, 0.003, 0.006, 0.01, 0.03, 0.06, 0.1, 0.3, 0.6, 1),
    cv=KFold(n_splits=10, random_state=None, shuffle=True),
    fit_intercept=True, gcv_mode=None, normalize=False, scoring=None,
    store_cv_values=False)

In [18]:
y_pred = rcv.predict(X_test_scaled)

# The coefficients
print('Coefficients: \n', rcv.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(y_test_scaled, y_pred))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test_scaled, y_pred))

Coefficients: 
 [[ 0.00842054  0.0156793   0.00293933 -0.00310603  0.00016025  0.00297598
   0.0018713  -0.00131867  0.00172717  0.00343741 -0.0016751  -0.00166654
   0.00012578 -0.00118315  0.00164353 -0.00566888  0.00013321  0.0002741
  -0.00650783 -0.00120694  0.00170048]]
Mean squared error: 0.99
Variance score: 0.00


In [ ]:
type(y_test_scaled)
y_pred

In [21]:
y_test_scaled

array([[ 0.28515592],
       [ 0.48037544],
       [-0.2684995 ],
       ...,
       [-0.8221549 ],
       [-0.48292094],
       [-0.00607323]], dtype=float32)